In [128]:
from torchnlp.datasets import penn_treebank_dataset
import torch
from torchnlp.samplers import BPTTBatchSampler
from torch.utils.data import DataLoader
from rsm_samplers import LangSequenceSampler
from ptb_lstm import LSTMModel
from lang_util import Corpus
import torch.nn.functional as F

In [75]:
corpus = Corpus('/Users/jgordon/nta/datasets/PTB')

[slice(0, 3, None),
 slice(185918, 185921, None),
 slice(371836, 371839, None),
 slice(557754, 557757, None),
 slice(743672, 743675, None)]

In [139]:
import importlib
import rsm_samplers
from torch.nn import CrossEntropyLoss
importlib.reload(rsm_samplers)

def _repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(_repackage_hidden(v) for v in h)


BS = 2
VS = 10000
sampler = rsm_samplers.LangSequenceSampler(corpus.train, batch_size=BS, seq_length=5, parallel_seq=True)

loader = DataLoader(corpus.train, batch_sampler=sampler,
                    collate_fn=rsm_samplers.language_pred_sequence_collate)

model = LSTMModel()
hidden = model.init_hidden(BS)
criterion = CrossEntropyLoss()

for i, (data, targets, _) in enumerate(loader):
    model.zero_grad()
    output, hidden, _ = model(data, hidden)
    loss = criterion(output.view(-1, VS), targets)
    loss.backward()
    print(loss.item())
    hidden = _repackage_hidden(hidden)
    if i > 5:
        break


9.214315414428711
9.19991683959961
9.21210765838623
9.22103214263916
9.222721099853516
9.200749397277832
9.220553398132324


In [4]:
len(corpus.dictionary)

10000

In [4]:
[len(corpus.train), len(corpus.valid), len(corpus.test)]

[929589, 73760, 82430]

In [6]:
batch_size = 300
batches = len(corpus.train) / batch_size
0.25 * batches

774.6575

In [12]:
from torch.nn.functional import softmax

def topk_mask(a, k, dim=0, do_softmax=False):
    """
    Return a 1 for the top b elements in the last dim of a, 0 otherwise
    """
    if do_softmax:
        return softmax(a)
    else:
        values, indices = torch.topk(a, k)
    arr = a.new_zeros(a.size())  # Zeros, conserve device
    arr.scatter_(dim, indices, 1)
    return arr

a = torch.randn((3, 4))
print(a)
topk_mask(a, 1, dim=1, do_softmax=True)

tensor([[ 1.3621,  0.2758, -0.5286,  0.0746],
        [ 0.8212,  0.9581, -0.0496, -0.1561],
        [ 1.4873, -0.6304,  1.5969,  1.2753]])


/Users/jgordon/miniconda3/envs/standard/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


tensor([[0.5668, 0.1913, 0.0856, 0.1564],
        [0.3400, 0.3898, 0.1423, 0.1279],
        [0.3284, 0.0395, 0.3664, 0.2657]])

In [65]:
from torch import nn
from copy import deepcopy
import matplotlib.pyplot as plt

class LocalLinear(nn.Module):
    """
    """
    def __init__(self, in_features, local_features, kernel_size, stride=1, bias=True):
        super(LocalLinear, self).__init__()
        self.kernel_size = kernel_size
        self.stride = stride

        fold_num = (in_features - self.kernel_size) // self.stride + 1
        self.lc = nn.ModuleList([deepcopy(nn.Linear(kernel_size, local_features, bias=bias))
                                 for _ in range(fold_num)])

    def forward(self, x):
        x = x.unfold(-1, size=self.kernel_size, step=self.stride)
        fold_num = x.shape[1]
        x = torch.cat([self.lc[i](x[:, i, :]) for i in range(fold_num)], 1)
        return x


class ActiveDendriteLayer(torch.nn.Module):
    """
    Local layer for active dendrites. Similar to a non-shared weight version of a 
    2D Conv layer.
    
    Note that dendrites are fully connected to input, local layer used only for connecting
    neurons and their dendrites
    """
    def __init__(self, input_dim, n_cells=50, n_dendrites=3):
        super(ActiveDendriteLayer, self).__init__()
        self.n_cells = n_cells
        self.n_dendrites = n_dendrites
        
        total_dendrites = n_dendrites * n_cells
        self.linear_dend = nn.Linear(input_dim, total_dendrites)
        self.linear_neuron = LocalLinear(total_dendrites, 1, n_dendrites, stride=n_dendrites)
        
    def __repr__(self):
        return "ActiveDendriteLayer neur=%d, dend per neuron=%d" % (self.n_cells, self.n_dendrites)
    
    def forward(self, x):
        x = F.relu(self.linear_dend(x))
        x = self.linear_neuron(x)
        return x
 
x = torch.randn(1, 5)
print(x)
adl = ActiveDendriteLayer(5, 4, 2)
print(adl(x))


tensor([[-0.2939,  0.5844,  0.6376, -0.2840, -0.3008]])
tensor([[ 0.3429, -0.1966, -0.4056, -0.0240]], grad_fn=<CatBackward>)


In [68]:
x = torch.randn(5, 3)
x[:, -2:] = 1
print(x)

tensor([[-0.1196,  1.0000,  1.0000],
        [-0.0664,  1.0000,  1.0000],
        [-0.3680,  1.0000,  1.0000],
        [ 1.5387,  1.0000,  1.0000],
        [ 1.1104,  1.0000,  1.0000]])


In [72]:
x.size()

torch.Size([5, 3])